 How many goals did the SU Women’s Lacrosse team score in 2023?



In [3]:
import pandas as pd

# Load your cleaned dataset
df = pd.read_csv("all_years_combined_player_stats.csv")
# First, drop rows where 'gp - gs' is null or doesn't contain a "-"
df = df[df['gp - gs'].notnull() & df['gp - gs'].str.contains('-')].copy()

# Now safely split and convert
df[['GP', 'GS']] = df['gp - gs'].str.split('-', expand=True)
df['GP'] = pd.to_numeric(df['GP'], errors='coerce').fillna(0).astype(int)
df['GS'] = pd.to_numeric(df['GS'], errors='coerce').fillna(0).astype(int)

# Drop the old columns if needed
df.drop(columns=['gp - gs', '#'], inplace=True, errors='ignore')

# Rename others if needed
df.rename(columns={
    'g': 'Goals',
    'A': 'Assists',
    'pts': 'Points',
    'sh': 'Shots',
    'sh%': 'Shot_Percentage',
    'sog': 'Shots_On_Goal',
    'sog%': 'SOG_Percentage',
    'gwg': 'Game_Winning_Goals',
    'FPG': 'Free_Position_Goals',
    'FPS': 'Free_Position_Shots',
    'GB': 'Ground_Balls',
    'to': 'Turnovers',
    'ct': 'Caused_Turnovers',
    'dc': 'Draw_Controls',
    'fouls': 'Fouls',
    'RC-YC-GC': 'Cards'
}, inplace=True)


How many goals did the SU Women’s Lacrosse team score in 2023?

In [22]:
df[df['Year'] == 2023]['Goals'].sum()

np.int64(337)

Who had the most assists in 2023?

In [23]:
df[df['Year'] == 2023].sort_values('Assists', ascending=False).iloc[0]

Player                 Ward, Emma
Goals                          38
Assists                        56
Points                         94
Shots                          73
Shot_Percentage             0.521
Shots_On_Goal                  48
SOG_Percentage              0.658
Game_Winning_Goals              1
Free_Position_Goals             7
Free_Position_Shots            12
Ground_Balls                    9
Turnovers                      30
Caused_Turnovers                1
Draw_Controls                   0
Fouls                          15
Cards                       0-2-0
Year                         2023
GP                             21
GS                              3
Name: 108, dtype: object

Who had the highest shooting accuracy (SOG%) overall?

In [21]:
df_filtered = df[df['Shots'] >= 10] 
top_sog = df_filtered.sort_values('SOG_Percentage', ascending=False).iloc[0]

print("Top SOG% Performer with Minimum 10 Shots:")
print(top_sog[['Player', 'Goals', 'Shots', 'SOG_Percentage']])

Top SOG% Performer with Minimum 10 Shots:
Player            Alexander, Morgan
Goals                             5
Shots                            14
SOG_Percentage                0.929
Name: 46, dtype: object


Who was the MVP in 2025 based on the Value_Score?

In [26]:
def compute_value_score(row):
    return (
        row['Goals'] * 2 +
        row['Assists'] * 1.5 +
        row['Game_Winning_Goals'] * 3 +
        row['SOG_Percentage'] * 5 -
        row['Turnovers']
    )

df['Value_Score'] = df.apply(compute_value_score, axis=1)

mvp_by_year = df.loc[df.groupby('Year')['Value_Score'].idxmax()][['Year', 'Player', 'Value_Score']]

In [28]:
df[df['Year'] == 2025].sort_values('Value_Score', ascending=False).head(1)

,Player,Goals,Assists,Points,Shots,Shot_Percentage,Shots_On_Goal,SOG_Percentage,Game_Winning_Goals,Free_Position_Goals,...,Ground_Balls,Turnovers,Caused_Turnovers,Draw_Controls,Fouls,Cards,Year,GP,GS,Value_Score
179,"Ward, Emma",30,46,76,77,0.39,55,0.714,1,3,...,6,41,2,0,9,0-1-0,2025,19,19,94.57


What was the average shooting accuracy of the team in 2021?

In [29]:
df[df['Year'] == 2021]['Goals'].sum() / df[df['Year'] == 2021]['Shots'].sum()

np.float64(0.48847926267281105)

Who was the most efficient scorer (highest Goals/Shots ratio) in 2025?

In [35]:
df['Shooting_Accuracy'] = df['Goals'] / df['Shots'].replace(0, 1) 


In [39]:
df['Year'] = df['Year'].astype(int)

In [59]:
df_2025 = df[(df['Year'] == 2025) & (df['Shots'] >= 10)]

most_efficient = df_2025.sort_values('Shooting_Accuracy', ascending=False).iloc[0]
print("Most Efficient Scorer in 2023 (min 10 shots):")
print(most_efficient[['Player', 'Goals', 'Shots', 'Shooting_Accuracy']])

Most Efficient Scorer in 2023 (min 10 shots):
Player               DeVito, Sam
Goals                          8
Shots                         12
Shooting_Accuracy       0.666667
Name: 190, dtype: object


In [60]:
df_2025.sort_values('Shooting_Accuracy', ascending=False)[['Player', 'Goals', 'Shots', 'Shooting_Accuracy']].head(5)


,Player,Goals,Shots,Shooting_Accuracy
190,"DeVito, Sam",8,12,0.666667
187,"Adamson, Olivia",10,18,0.555556
182,"Britton, Gracie",20,41,0.487805
181,"Muchnick, Emma",34,71,0.478873
191,"Parker, Annie",6,13,0.461538


Who was the top goal scorer in 2025?

In [4]:
df[df['Year'] == 2025].sort_values('Goals', ascending=False).iloc[0]['Player']

'Muchnick, Emma'

Which player had the highest GameChanger Score in 2024?

In [8]:
df['GameChanger_Score'] = (
    df['Goals'] * 2 +
    df['Assists'] * 1.5 +
    df['Game_Winning_Goals'] * 3 +
    df['SOG_Percentage'] * 5 -
    df['Turnovers']
)

In [9]:
df[df['Year'] == 2024].sort_values('GameChanger_Score', ascending=False).iloc[0]['Player']

'Tyrrell, Emma'

Who was the most improved player from 2020 to 2025?

In [10]:
# Average points by player by year
pivot = df.pivot_table(index='Player', columns='Year', values='Points', aggfunc='sum')

# Calculate improvement from earliest to latest year (or use specific years like 2020 to 2025)
pivot['Improvement'] = pivot[2025].fillna(0) - pivot[2020].fillna(0)


# Top improved player
pivot.sort_values(by='Improvement', ascending=False).head(1)

Year,2020,2021,2022,2023,2024,2025,Improvement
Player,,,,,,,
"Ward, Emma",NaN,73.0,NaN,94.0,81.0,76.0,76.0


If the coach wants to win 2 more games in 2025, should they focus on offense or defense? Which player could help most?

In [22]:
# Filter for 2025 season
df_2025 = df[df['Year'] == 2025].copy()

# Calculate total team offense stats
total_goals = df_2025['Goals'].sum()
total_assists = df_2025['Assists'].sum()
avg_sog = df_2025['SOG_Percentage'].mean()
total_turnovers = df_2025['Turnovers'].sum()

print(f"\n🏹 2025 Offense Summary:")
print(f"Total Goals: {total_goals}")
print(f"Total Assists: {total_assists}")
print(f"Average Shooting Accuracy (SOG%): {avg_sog:.3f}")
print(f"Total Turnovers: {total_turnovers}")

# Find most efficient players (SOG% > 0.75 and Goals > 5)
efficient_scorers = df_2025[(df_2025['SOG_Percentage'] > 0.75) & (df_2025['Goals'] > 5)]
print("\n🔥 High-Efficiency Scorers in 2025 (SOG% > 0.75 & Goals > 5):")
print(efficient_scorers[['Player', 'Goals', 'SOG_Percentage']])

# Find best playmaker (most assists)
top_playmaker = df_2025.sort_values(by='Assists', ascending=False).iloc[0]
print(f"\n🎯 Top Playmaker: {top_playmaker['Player']} with {top_playmaker['Assists']} assists")

# Find player with most turnovers (defensive concern)
most_turnovers = df_2025.sort_values(by='Turnovers', ascending=False).iloc[0]
print(f"\n⚠️ Highest Turnovers: {most_turnovers['Player']} with {most_turnovers['Turnovers']} turnovers")

# GameChanger suggestion
print("\n🏆 Suggestion:")
print(f"Focus on maximizing offensive output by coordinating between {top_playmaker['Player']} and high-efficiency finishers like:")
for player in efficient_scorers['Player'].tolist():
    print(f" - {player}")


🏹 2025 Offense Summary:
Total Goals: 234
Total Assists: 112
Average Shooting Accuracy (SOG%): 0.486
Total Turnovers: 255

🔥 High-Efficiency Scorers in 2025 (SOG% > 0.75 & Goals > 5):
                 Player  Goals  SOG_Percentage
180  Trinkaus, Caroline     32           0.792
181      Muchnick, Emma     34           0.775
182     Britton, Gracie     20           0.805
183     Vogelman, Alexa     21           0.761
184     Cotter, Mileena     21           0.760
190         DeVito, Sam      8           0.917

🎯 Top Playmaker: Ward, Emma with 46 assists

⚠️ Highest Turnovers: Ward, Emma with 41 turnovers

🏆 Suggestion:
Focus on maximizing offensive output by coordinating between Ward, Emma and high-efficiency finishers like:
 - Trinkaus, Caroline
 - Muchnick, Emma
 - Britton, Gracie
 - Vogelman, Alexa
 - Cotter, Mileena
 - DeVito, Sam
